# ISYS613 - Data Sourcing and Quality
## Assignment 3
## Web APIs

## Question_1
The Consumer Complaint Database is a collection of complaints about
 consumer financial products and services that are sent to companies for
 response. Complaints are published after the company responds,
 confirming a commercial relationship with the consumer, or after 15
 days, whichever comes first.

### Field reference
Refer to the following URL for a complete listing and description of the fields are currently included
in the database. https://cfpb.github.io/api/ccdb/fields.html

### Direct Data Download
A recent version of the Consumer Complaint Database is available for direct
download from the following URL.

https://www.drivehq.com/file/DFPublishFile.aspx/FileID7603336590/Keyitc9ddc81b6i/complaints_sm.csv

### Programming Tasks
Author the python code necessary to answer the following questions:

1. What is the most common type of consumer identified 'product' in the complaint database?
2. Considering only the most common type of consumer identified 'product' in the complaint database,
what percentage of those complaints were associated with a timely response by the company identified in
the complaint?

In [1]:
import requests
import json
from requests.exceptions import HTTPError
import csv


def url_request(url:str, *args, **kwargs) -> requests.Response:
    try:
        response = requests.get(url, *args, **kwargs)
        # If the response was successful, no Exception will be raised
        # by the following line
        response.raise_for_status()

    except HTTPError as http_err:
        # Cannot proceed
        print(f'HTTP error occurred: {http_err}')
        raise
    except Exception as err:
        # Cannot proceed
        print(f'Other error occurred: {err}')
        raise
    else:
        print(f'Success: {response.status_code}')

    return response.text


def txt_csv_dict(data):
    
    csv_data = data.split('\n')
    reader = csv.DictReader(csv_data)
    return reader
    
def get_most_common_product(reader):
    
    product_counts = {}
    
    for row in reader:
        product = row['Product']
        if product in product_counts:
            product_counts[product] += 1
        else:
            product_counts[product] = 1

   
    most_common_product = max(product_counts, key=product_counts.get)
    return most_common_product


def calculate_timely_response_percentage(reader, product):
    
    total_complaints = 0
    timely_responses = 0
    
    for row in reader:
        
        if row['Product'] == product:
            total_complaints += 1
            
            if row['Timely response?'] == 'Yes':
                timely_responses += 1

    percentage = (timely_responses / total_complaints) * 100 if total_complaints > 0 else 0
    return percentage


if __name__ == '__main__':
    # TEST DATA
    URL = 'https://www.drivehq.com/file/DFPublishFile.aspx/FileID7603336590/Keyitc9ddc81b6i/complaints_sm.csv'
   
    data = url_request(URL)
    reader = list(txt_csv_dict(data))
    
    most_common_product = get_most_common_product(reader)
    timely_response_percentage = calculate_timely_response_percentage(reader, most_common_product)
    
    print(f"The most common product is: {most_common_product}")
    print(f"The percentage of timely responses for this product is: {timely_response_percentage:.2f}%")


Success: 200
The most common product is: Credit reporting, credit repair services, or other personal consumer reports
The percentage of timely responses for this product is: 98.21%


## Question_2

The US Federal Government maintains many energy related Web-based resources.
One such resource provides programmatic access to several renewable energy data categories.
In particular, there is an API endpoint that supports access to a database of
alternative fuel vehicles (AFVs), manufacturers, fuels, engines, and hybrid/conversion systems.

In this question you will query this API in order to return information regarding alternative fuel
light-duty vehicles.

#### API Documentation
The alternative fuel vehicle (AFV) API documentation portal uses the OpenAPI Specification.
The OpenAPI Specification (originally known as the Swagger
 Specification) is a specification for machine-readable interface files
 for describing, producing, consuming, and visualizing RESTful web services.

The OpenAPI documentation for AFVs can be seen at:

https://developer.nrel.gov/docs/transportation/vehicles-v1/

### Request Data Requirements
Use one or more of the endpoints documented by the docs URL above to answer the following questions.

NOTE: To ensure coverage of the assignment related material, some of the questions have specific requirements
regarding the output_format to be requested when retrieving the payload data.

### Programming Tasks
Author the python code necessary to answer the following questions:

1. Which light-duty automobile manufacturer (by name) offered the most light-duty AFVs in the model year 2020?
Required *output_format*: XML.
**Hint:** Look carefully at the /v1/light_duty_automobiles.{output_format}

2. Which alternative fuels types are supported by the most current vehicle categories?
Return only fuels associated with current light-duty automobiles
Required *output_format*: JSON.
**Hint:** Look carefully at the /v1/fuels.{output_format} endpoint.

In [4]:
import requests
import json
import xml.etree.ElementTree as ET

# Define the API base URL
base_url = "https://developer.nrel.gov/api/vehicles"

# Replace with your API key if required
api_key = "7iBnQl5GMBgun0YcmqG1N2k4gfK9jwvi7klwdalv"

# Helper function to make API requests
def make_api_request(endpoint, params=None):
    headers = {"Content-Type": "application/json"}
    if api_key:
        params = params or {}
        params["api_key"] = api_key

    response = requests.get(f"{base_url}{endpoint}", params=params, headers=headers)
    response.raise_for_status()
    
    return response

# Question 1: Find the manufacturer with the most light-duty AFVs in the model year 2020 (Output format: XML)
def get_manufacturer_with_most_afvs():
    endpoint = f"/v1/light_duty_automobiles.xml"
    response = make_api_request(endpoint)
    
    root = ET.fromstring(response.text)
    print(root)
    max_afvs = 0
    manufacturer_name = ""

    for vehicle in root.findall(".//vehicle"):
        model_year = vehicle.find("model_year").text
        if model_year == "2020":
            afvs_count = len(vehicle.findall(".//afvs/afv"))
            if afvs_count > max_afvs:
                max_afvs = afvs_count
                manufacturer_name = vehicle.find("make").text

    return manufacturer_name

# Question 2: Find alternative fuel types supported by the most current light-duty vehicle categories (Output format: JSON)
def get_fuels_supported_by_current_categories():
    endpoint = "/v1/fuels.json"
    response = make_api_request(endpoint)
    fuels_data = response.json()
    
    
    fuels_by_category = []
    
    for fuel_category in fuels_data["result"]:
        if fuel_category["has_light_duty_automobiles?"]:
            fuels_by_category.append(fuel_category["name"])
        #else:
            #print(fuel_category["name"])


    return fuels_by_category



# Question 1
manufacturer_with_most_afvs = get_manufacturer_with_most_afvs()
print("Manufacturer with the most light-duty AFVs in the model year 2020:", manufacturer_with_most_afvs)

# Question 2
fuels_supported_by_current_categories = get_fuels_supported_by_current_categories()
print("Alternative fuels supported by the most current light-duty vehicle categories:", fuels_supported_by_current_categories)


<Element 'response' at 0x7fb4d3c6f560>
Manufacturer with the most light-duty AFVs in the model year 2020: 
Alternative fuels supported by the most current light-duty vehicle categories: ['Methanol', 'Biodiesel (B20)', 'Propane', 'Electric', 'Hybrid Electric', 'Ethanol (E85)', 'Hydrogen Fuel Cell', 'CNG - Compressed Natural Gas', 'CNG - Bi-fuel', 'Propane - Bi-fuel', 'Plug-in Hybrid Electric']
